In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
warnings.simplefilter("ignore")
from datetime import datetime
import plotly.graph_objects as go
from skimage import io
import plotly.express as px
import time

In [2]:
Sunday = pd.DataFrame(pd.read_csv('park-movement-Sun_New.csv'))
Sunday = Sunday.drop(["Unnamed: 0"], axis=1)
Sunday["id"] = Sunday["id"].astype(str)
Sunday["Facility_coordinates"]= Sunday.apply(lambda row: (int(row.X),int(row.Y)), axis=1)
Sunday["X"] = Sunday["X"].map(lambda x:int(x))
Sunday["Y"] = Sunday["Y"].map(lambda x:int(x))

In [3]:
Sunday["id"] = Sunday["id"].map(lambda x: x.split(".", 1)[0])

In [4]:
Sunday

,Timestamp,id,type,X,Y,visit_count,visit_duration,Facility_coordinates
0,1402236011,1923259,1,0,67,1418,765,"(0, 67)"
1,1402236046,1923259,0,1,66,1418,765,"(1, 66)"
2,1402236054,1923259,0,2,67,1418,765,"(2, 67)"
3,1402236062,1923259,0,3,67,1418,765,"(3, 67)"
4,1402236070,1923259,0,4,67,1418,765,"(4, 67)"
...,...,...,...,...,...,...,...,...
10932419,1402290841,571266,0,95,77,475,327,"(95, 77)"
10932420,1402290848,571266,0,96,77,475,327,"(96, 77)"
10932421,1402290855,571266,0,97,76,475,327,"(97, 76)"
10932422,1402290862,571266,0,98,77,475,327,"(98, 77)"


In [6]:
d1 = pd.read_csv('SundayGroupCount.csv')
d2 = pd.read_csv('SundayCrowdCount.csv')
d3 = pd.read_csv('SundaySingleCount.csv')

Group = [  i.split(".", 1)[0] for i in d1.columns ]
Crowd = [  i.split(".", 1)[0] for i in d2.columns ]
Single = [  i.split(".", 1)[0] for i in d3.columns ]

Sunday_Group = list(set(Group))
Sunday_Crowd = list(set(Crowd)-set(Group))
Sunday_Single=  list(set(list(set(Single)-set(Group)))-set(Crowd)) 

In [9]:
TimePointMorning = time.mktime(time.strptime("2014-6-08 11:00:00", "%Y-%m-%d %H:%M:%S"))
TimePointAfterNoon = time.mktime(time.strptime("2014-6-08 16:00:00", "%Y-%m-%d %H:%M:%S"))

ArrivalTime = Sunday.copy().drop_duplicates(subset='id')
ArrivalTime["ArrivalTime"] = ArrivalTime["Timestamp"].map(lambda x: 1 if x- TimePointMorning >= 0 and x - TimePointAfterNoon <= 0 else ( 2 if x - TimePointAfterNoon >= 0 else 0 ) )
Early = list(ArrivalTime[ArrivalTime["ArrivalTime"]==0].id.unique())
#Early = [  i.split(".", 1)[0] for i in Early ]
Noon =  list(ArrivalTime[ArrivalTime["ArrivalTime"]==1].id.unique())
#Noon = [  i.split(".", 1)[0] for i in Noon ]
Late = list(ArrivalTime[ArrivalTime["ArrivalTime"]==2].id.unique())
#aLate = [  i.split(".", 1)[0] for i in Late ]

In [10]:
m = np.mean(Sunday["visit_duration"].unique())

Sunday_IITime = Sunday.copy().drop_duplicates(subset='id')
HighDuration = list(Sunday_IITime[Sunday_IITime["visit_duration"] >= m ].id)
#HighDuration = [  i.split(".", 1)[0] for i in HighDuration ]
LowDuration = list(Sunday_IITime[Sunday_IITime["visit_duration"] < m ].id)
#LowDuration = [  i.split(".", 1)[0] for i in LowDuration ]

In [52]:
np.mean(Sunday["visit_duration"].unique())

594.7958115183246

### Group 1

In [14]:
G1 = list(set(Sunday_Group)&set(Early)&set( HighDuration))
G1P = Sunday.loc[Sunday['id'].isin(G1)]
len(G1), round(len(G1)/len(Sunday.id.unique()),3)

(5212, 0.689)

In [13]:
GroupV = pd.DataFrame(G1P[G1P.type == 1].groupby(["X","Y"])["id"].count())
GroupV = GroupV.reset_index()
GroupV.columns = ["X","Y","count"]
GroupV.sort_values(by=['count'],ascending=False)[0:5]

,X,Y,count
12,38,90,5695
36,86,44,5573
8,27,15,5480
17,47,11,5460
16,45,24,5404


### Group 2

In [42]:
G2 = list(set(Sunday_Crowd)&set(Early)&set( HighDuration))
G2P = Sunday.loc[Sunday['id'].isin(G2)]
len(G2), round(len(G2)/len(Sunday.id.unique()),3)

(913, 0.121)

In [43]:
GroupV = pd.DataFrame(G2P[G2P.type == 1].groupby(["X","Y"])["id"].count())
GroupV = GroupV.reset_index()
GroupV.columns = ["X","Y","count"]
GroupV.sort_values(by=['count'],ascending=False)[0:5]

,X,Y,count
12,38,90,1143
16,45,24,1063
36,86,44,1047
17,47,11,976
8,27,15,967


### Group 3

In [17]:
G3 = list(set(Sunday_Single)&set(Early)&set(HighDuration))
G3P = Sunday.loc[Sunday['id'].isin(G3)]
len(G3), round(len(G3)/len(Sunday.id.unique()),3)

(875, 0.116)

In [18]:
GroupV = pd.DataFrame(G3P[G3P.type == 1].groupby(["X","Y"])["id"].count())
GroupV = GroupV.reset_index()
GroupV.columns = ["X","Y","count"]
GroupV.sort_values(by=['count'],ascending=False)[0:5]

,X,Y,count
12,38,90,1087
36,86,44,1036
17,47,11,1036
8,27,15,1028
16,45,24,1023


### Group 4

In [36]:
G4 = list(set(Sunday_Group)&set(Noon))
G4P = Sunday.loc[Sunday['id'].isin(G4)]
len(G4), round(len(G4)/len(Sunday.id.unique()),3) 

(60, 0.008)

In [37]:
GroupV = pd.DataFrame(G4P[G4P.type == 1].groupby(["X","Y"])["id"].count())
GroupV = GroupV.reset_index()
GroupV.columns = ["X","Y","count"]
GroupV.sort_values(by=['count'],ascending=False)[0:5]

,X,Y,count
32,86,44,80
10,38,90,58
18,63,99,49
7,27,15,39
14,45,24,33


### Group 5

In [38]:
G5 = list(set(Sunday_Crowd)&set(Noon))
G5P = Sunday.loc[Sunday['id'].isin(G5)]
len(G5), round(len(G5)/len(Sunday.id.unique()),3) 

(8, 0.001)

In [39]:
GroupV = pd.DataFrame(G5P[G5P.type == 1].groupby(["X","Y"])["id"].count())
GroupV = GroupV.reset_index()
GroupV.columns = ["X","Y","count"]
GroupV.sort_values(by=['count'],ascending=False)[0:5]

,X,Y,count
28,86,44,8
16,63,99,7
30,87,63,7
23,79,89,5
8,38,90,5


### Group 6

In [23]:
G6 = list(set(Sunday_Single)&set(LowDuration))
G6P = Sunday.loc[Sunday['id'].isin(G6)]
len(G6) , round(len(G6)/len(Sunday.id.unique()),3)

(127, 0.017)

In [24]:
GroupV = pd.DataFrame(G6P[G6P.type == 1].groupby(["X","Y"])["id"].count())
GroupV = GroupV.reset_index()
GroupV.columns = ["X","Y","count"]
GroupV.sort_values(by=['count'],ascending=False)[0:5]

,X,Y,count
12,38,90,133
33,86,44,133
17,47,11,126
8,27,15,115
16,45,24,112


### Group 7

In [29]:
G7 = list(set(Sunday_Group)&set(LowDuration)&set(Early))
G7P = Sunday.loc[Sunday['id'].isin(G7)]
len(G7), round(len(G7)/len(Sunday.id.unique()),3)

(335, 0.044)

In [30]:
GroupV = pd.DataFrame(G7P[G7P.type == 1].groupby(["X","Y"])["id"].count())
GroupV = GroupV.reset_index()
GroupV.columns = ["X","Y","count"]
GroupV.sort_values(by=['count'],ascending=False)[0:5]

,X,Y,count
16,45,24,253
21,63,99,224
36,86,44,200
17,47,11,184
8,27,15,159


### Group 8

In [27]:
G8 = list(set(Sunday_Group)&set(Late))
G8P = Sunday.loc[Sunday['id'].isin(G8)]
len(G8), round(len(G8)/len(Sunday.id.unique()),3)

(21, 0.003)

In [28]:
GroupV = pd.DataFrame(G8P[G8P.type == 1].groupby(["X","Y"])["id"].count())
GroupV = GroupV.reset_index()
GroupV.columns = ["X","Y","count"]
GroupV.sort_values(by=['count'],ascending=False)[0:5]

,X,Y,count
13,78,48,16
12,78,37,13
20,99,77,12
18,87,48,12
10,63,99,9


In [44]:
g1 = pd.read_csv('SundayGapID.csv')
g2 = pd.read_csv('SundayGapMax.csv')
g3 = pd.read_csv('SundayGapMed.csv')

GapID = [  i.split(".", 1)[0] for i in g1.columns ]
GapMax = [  i.split(".", 1)[0] for i in g2.columns ]
GapMed = [  i.split(".", 1)[0] for i in g3.columns ]

In [45]:
inds1 =  np.argsort(GapMax)
GapH = np.array(GapID)[inds1[-150:]]
GapHV = Sunday.loc[Sunday['id'].isin(GapH)]

GapHV2 = pd.DataFrame(GapHV[GapHV.type == 1].groupby(["X","Y"])["id"].count())
GapHV2 = GapHV2.reset_index()
GapHV2.columns = ["X","Y","count"]

In [46]:
GapHV2.sort_values(by=['count'],ascending=False)[0:5]

,X,Y,count
12,38,90,154
16,45,24,115
28,78,48,104
35,86,44,102
8,27,15,102


In [47]:
inds2 =  np.argsort(GapMed) 
Faster = np.array(GapID)[inds2[0:150]]
FasterV = Sunday.loc[Sunday['id'].isin(Faster)]

FasterV2 = pd.DataFrame(FasterV[FasterV.type == 1].groupby(["X","Y"])["id"].count())
FasterV2 = FasterV2.reset_index()
FasterV2.columns = ["X","Y","count"]

In [48]:
FasterV2.sort_values(by=['count'],ascending=False)[0:5]

,X,Y,count
36,86,44,266
16,45,24,197
17,47,11,183
12,38,90,178
8,27,15,147


In [49]:
inds3 =  np.argsort(GapMed)
Slower = np.array(GapID)[inds3[-150:]]
SlowerV = Sunday.loc[Sunday['id'].isin(Slower)]

SlowerV2 = pd.DataFrame(SlowerV[SlowerV.type == 1].groupby(["X","Y"])["id"].count())
SlowerV2 = SlowerV2.reset_index()
SlowerV2.columns = ["X","Y","count"]

In [50]:
SlowerV2.sort_values(by=['count'],ascending=False)[0:5]

,X,Y,count
17,47,11,259
36,86,44,248
12,38,90,216
8,27,15,170
16,45,24,163
